<a href="https://colab.research.google.com/github/GimenesPaula/GimenesPaula/blob/main/Projeto%201.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fazer Upload planilhas
1.   Launches
2.   Inci
3.   Sales
4.   Project

In [ ]:
#Launches
from google.colab import files
uploaded = files.upload()

Saving 2024.xlsx to 2024.xlsx


In [ ]:
filename = next(iter(uploaded))

In [ ]:
# INCI name produtos
from google.colab import files
inci = files.upload()

Saving inci.xlsx to inci.xlsx


In [ ]:
filename2 = next(iter(inci))

In [ ]:
#Projetos Distribuidores
from google.colab import files
proj = files.upload()

Saving Projetos.xlsx to Projetos.xlsx


In [ ]:
filename3 = next(iter(proj))

In [ ]:
#Vendas Distribuidores
from google.colab import files
dist = files.upload()

Saving Vendas.xlsx to Vendas.xlsx


In [ ]:
filename4 = next(iter(dist))

# Bibliotecas Phyton

In [ ]:
#Carrega Bibliotecas
import pandas as pd
import numpy as np
import re

In [ ]:
pip install unidecode

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 6.0 MB/s eta 0:00:00


In [ ]:
from unidecode import unidecode

##Carrega tabela do Mintel

In [ ]:
#Carrega o banco de dados como tabela
data = '/content/'+filename
df_launches = pd.read_excel(data)
pd.options.display.max_columns = None
df_launches.nunique()

Record ID                       1697
Date Published                   103
Product                         1143
Brand                           1160
Company                          481
Market                             1
Category                           4
Sub-Category                      38
Product Description             1696
Launch Type                        5
Manufacturer                     434
Location of Manufacture           23
Manufacturer Company Address     309
Ingredients (Standard form)     1667
Price in local currency          598
Primary Image Link              1697
Pet                                0
Convenience                       20
Minus                              0
Functional                        74
Natural                           24
Plus                               2
Positioning                       15
Suitable for                       6
Demographic                        8
Free from                         49
Beauty enhancing                 106
E

##Carrega Tabela de INCI

In [ ]:
data = '/content/'+filename2
df_inci = pd.read_excel(data)
pd.options.display.max_columns=None
df_inci.nunique()

Produto        52
Ingrediente    48
Prioridade      3
dtype: int64

##Carrega Tabela de Projetos

In [ ]:
data = '/content/'+filename3
df_proj = pd.read_excel(data)
pd.options.display.max_columns=None
df_proj.nunique()

Distributor      4
Customer       628
UF              21
Material       201
dtype: int64

## Carrega Planilha de Vendas do Distribuidor

In [ ]:
data = '/content/'+filename4
df_dist = pd.read_excel(data)
pd.options.display.max_columns=None
df_dist.nunique()

Customer(at distrib)    1819
Country, Customer          1
Distributor                3
Material                 130
Year                     404
dtype: int64

# Define funções

In [ ]:
#this checks if any combination of INCI as present in Ingredient
def verifica_ingrediente(formula,produto,material):
  quantidade = len(produto.difference(formula))
  if quantidade == 0:
    return material
  return None

In [ ]:
#If last code is true, this returns the material name
def procura_produtos(formula, produtos, materiais):
  formula = formula.copy()
  resultados = []
  for prod, mat in zip(produtos, materiais):
    resultado = verifica_ingrediente(formula, prod, mat)
    if resultado is not None:
      formula = formula.difference(prod) ## Para remover os ingredientes já encontrados numa nova busca.
      resultados.append(resultado)
  return resultados

In [ ]:
#this code transpose data. Used when we bring each category and the number of lauches.
def transpor (df, coluna, linha):
  for cat in df[coluna].unique():
    f = df[coluna] == cat
    df[cat] = df[f][linha]
    f = df[cat].isna()
    df.loc[f, cat] = df.loc[f, cat].apply(lambda x:[])

In [ ]:
#Code to solve an issue from algoritms
def to_set(x):
    try:
        if isinstance(x, pd.DataFrame):
            return x.apply(to_set)
        else:
            return set(x.sum())
    except:
        pass

In [ ]:
#ode to solve an issue from algoritms
def to_set_2(x):
    try:
        if isinstance(x, pd.DataFrame):
            return x.apply(to_set_2)
        else:
            return list(set(x))
    except:
        pass

In [ ]:
#This is a dictionary of words that may be removed from Company, Manufacture
#in order to group manufacture despite of spelling mistakes
def formatar_fabricantes(coluna):
  remover = [
             'fabrica', 'fabricacao', 'manuf', 'tercerizacao', 'manufacturing',
             'farmacia', 'farmacias', 'farmaceuticos', 'drogaria','drogarias', 'farm',
             'cosmetico', 'cometico','cosmeticos', 'cometicos', 'cosmetics',
             'cosmiatria', 'cosmetica', 'cosmeticas', 'cosm', 'cosme', 'kosmetics',
             'ltda','epp', 'eireli', 'me', 'eirell', 'sa', 'eir', 'ltd',
             'indústria','industria', 'industrias', 'industrial', 'indutrial', 'ind', 'industriae',
             'comercio', 'comecio', 'comercial', 'corporation', 'com',
             'imp', 'exp',  'prod', 'exportacao','importacao','importadora', 'import',
             'do', 'e', 'para', 'de', 'le', 'la',
             'do brasil', 'aerossois', 'aerosol', 'aerossol', 'tecnologia',
             'laboratorio','laboratorios', 'laboratories', 'laboratoire', 'instituto', 'group',
             'pesquisa', 'desenvolvimento', 'naturais', 'professional', 'aromatizantes',
             'produtos', 'Produto', 'saneantes', 'quimica', 'quimicos', 'quimico', 'farmaceutica', 'veterinário',
             'higiene','pessoal', 'limpeza', 'beleza', 'botanico',
             'perfumes','fragrancias', 'perfumaria', 'medicamentos', 'perf',
             'distribuidora', 'dist',
             'international', 'internacional', 'sulamericana', 'nacional',
             'nbsp;', 'veer','serv', 'vajerista',
           ]
  pat = '\\b({})\\b'.format('|'.join(remover))
  coluna = (
    coluna
    .apply(unidecode)
    .str.lower()
    .replace('\.', ' ',regex=True)
    .replace('\/', '',regex=True)
    .replace('\-', '',regex=True)
    .replace(pat,'',regex=True)
    .replace(',', '',regex=True)
    .replace('\s+','',regex=True) #Tirar numeros
    .str.strip()
  )
  return coluna

In [ ]:
#Dis dict clean the name of material
def formatar_material(coluna):
  remover = ['BELSIL®', 'BELSIL', 'BELSIM',
             'DRUM', 'ST', 'PE', 'CANISTER', 'RD', 'CARTON', 'IBC PE', '2', 'WOPAL',
             'SBPAL', 'STPAL', 'HOBBOCK', 'SACK', 'BOTTLE',
             '0,45KG','10KG','15KG','25KG', '30G', '60G', '100KG', '140KG', '150KG', '160KG',
             '180KG', '190KG', '200KG', '210KG', '900KG', '950KG', '190',
             'KG', 'AMOSTRA', 'PA', 'XY', 'CP', 'SILICONE ELASTOMER GEL',
             'BR', 'DE', ' E', 'BLOCKED',
             'EMB. 0,100 KG', 'Ver  Pf 100', 'USP',
           ]
  pat = '\\b({})\\b'.format('|'.join(remover))
  coluna = (
    coluna
    .replace(pat,'',regex=True)
    .replace('\#','', regex=True)
    .replace('\"','', regex=True)
    .str.replace('\/', '',regex=True)
    .str.replace('\-', '',regex=True)
    .str.replace('\*', '',regex=True)
    .str.title()
    .str.strip()
  )
  return coluna

# Mintel

## Define Estados


In [ ]:
dict = {'AC':'AC', 'AL':'AL', 'AM':'AM', 'AP':'AP', 'BA':'BA', 'CE':'CE', 'DF':'DF', 'ES':'ES', 'GO':'GO', 'MA':'MA',
     'MG':'MG', 'MS':'MS', 'MT':'MT', 'PA':'PA', 'PB':'PB', 'PE':'PE', 'PI':'PI', 'PR':'PR', 'RJ':'RJ', 'RN':'RN',
     'RO':'RO', 'RR':'RR', 'RS':'RS', 'SC':'SC', 'SE':'SE', 'SP':'SP', 'TO':'TO', 'Sao Paulo':'SP', 'São Paulo':'SP',
     'Sao Paolo':'SP','Osasco': 'SP', 'Mogi das Cruzes': 'SP', 'Cotia':'SP', 'Diadema':'SP', 'São Roque':'SP',
     'São Sebastião':'SP', 'Lins':'SP', 'Cajamar':'SP', 'Pirassununga':'SP', 'Paulista': 'SP', 'São Paolo':'SP',
     'Elisario':'SP', 'Valinhos':'SP', 'Piracaia':'SP', 'São José do Rio Preto':'SP', 'Sao José do Rio Preto':'SP',
     'São José dos Campos':'SP',      'Araras':'SP', 'São Bernardo do Campo':'SP', 'São José dos Campos':'SP',
     'Sao Caetano do Sul':'SP', 'São José do Rio Preto':'SP', 'Brodowski':'SP', 'Cajuru':'SP', 'Itaquaquecetuba':'SP',
     'Itupeva':'SP', 'SO':'SP', 'Mairiporã':'SP', 'Itaquera':'SP', 'Franco da Rocha':'SP', 'Guarulhos':'SP',
     'Bairro Jardim America':'SP', 'Jardim Brazil':'SP', 'Agudos':'SP', 'Itapira':'SP', 'Rib. Preto':'SP', 'Vinhedo':'SP',
     'Lorena':'SP', 'Caieiras': 'SP', 'Louveira':'SP', 'Taboao da Serra':'SP', 'Hortôlandia':'SP', 'Americana':'SP',
     'Macatuba':'SP', 'Boituva':'SP', 'Jandira':'SP', 'Bom Jesus':'SP', 'Atibaia':'SP',
     'Ribeirão Preto':'SP', 'Cabreúva': 'SP', 'Presidente Prudente':'SP', 'Jurubatuba':'SP',
     'Jaú':'SP',
     'Rio de Janeiro':'RJ', 'Rio De Janeiro':'RJ', 'Nova Iguaçu': 'RJ', 'Nova Iguacu': 'RJ', 'Nivopolis':'RJ',
     'Duque de Caixas':'RJ', 'Engenheiro Pedreira':'RJ', 'Areal':'RJ', 'Duque de Caxias':'RJ',
     'Vila Velha':'ES', 'Marechal Floriano':'ES', 'Cariacica':'ES', 'Bom Pastor':'ES',
     'Belo Horizonte':'MG', 'Minas Gerais':'MG', 'Pocos de Caldas':'MG', 'Viçosa':'MG', 'Vicosa':'MG',
     'Pouso Alegre': 'MG', 'Uberlândia':'MG', 'Uberlandia':'MG', 'Ipatinga':'MG', 'Mirabela':'MG','Nova Lima':'MG',
     'São José da Lapa':'MG', 'Sao José da Lapa':'MG', 'Sao Jose da Lapa':'MG', 'Divinópolis':'MG',
     'Tres Pontas':'MG', 'Castelo BH':'MG', 'Varginha':'MG', 'Elói Mendes':'MG', 'Sete Lagoas':'MG',
     'Lagoa Santa':'MG',
     'Porto Alegre':'RS', 'Estância Velha': 'RS', 'Caxias do Sul':'RS', 'Passo Fundo':'RS', 'Gravatai':'RS',
     'Panambi':'RS', 'Gramado':'RS', 'Roca Sales':'RS',
     'Santa Catarina':'SC', 'Joinville':'SC','Camboriu':'SC', 'Maluche':'SC', 'Araquari':'SC', 'Criciúma': 'SC',
     'Criciuma': 'SC', 'Chapecó':'SC', 'Três Barras': 'SC', 'Palhoça':'SC', 'Palhoca':'SC',
     'Colombo':'PR', 'Paraná':'PR', 'Curitiba':'PR', 'Sao José dos Pinhais':'PR','BR 116 Km 85':'PR',
     'Cambé':'PR', 'Fazenda Rio Grande':'PR', 'Mandaguaçu':'PR', 'Pinhais':'PR',
     'Goiana':'GO', 'Goiânia':'GO', 'Anápolis':'GO', 'Anapolis':'GO', 'Go':'GO', 'Itapuranga': 'GO',
     'Guapó':'GO', 'Itajaí':'GO',
     'Brasília': 'DF', 'Brasilia': 'DF',
     'Rondônia':'RO','Novo Hamburgo':'RO','Manaus':'AM', 'Pará':'PA',
     'Bahia':'BA', 'Lauro de Freites': 'BA', 'Lauro de Freitas': 'BA', 'Av. Queira Deus':'BA',
     'Vila Cia Sul':'BA', 'Juazeiro':'BA',
     'Recife':'PE', 'Camaragibe': 'PE', 'Abreu e Lima':'PE', 'Fortaleza':'CE', 'Ceará':'CE', 'Caucaia':'CE',
     'Aracaju': 'SE', 'Nossa Sra. do Socorro':'SE',
     }

In [ ]:
df_launches['State'] = df_launches['Manufacturer Company Address'].str.extract('('+'|'.join(dict)+')',expand=False).map(dict)
#Exibe a quantidade de casos únicos
df_launches['State'].value_counts(dropna=False)
#Em output se filtra estados vazios

State
SP     789
NaN    374
MG     108
RJ      91
PR      74
GO      64
RS      64
SC      42
CE      20
BA      18
ES      13
PE      11
DF      11
SE       9
RO       6
AM       2
PA       1
Name: count, dtype: int64

## Search dictOTHERS words in the ingredient, in order to identify cases where the words occure.

In [ ]:
#Dicionário Silicones geral
dictOTHERS = {'methicone':'1', 'Dimethicone':'1','methicone Crosspolymer':'1','methiconol':'1',
              'ylsiloxysilicate':'1', 'ylsilsesquioxane':'1', 'Disiloxane':'1', 'Silica':'1','siloxane':'1'}

In [ ]:
df_launches['Silicone'] = df_launches['Ingredients (Standard form)'].str.extract('('+'|'.join(dictOTHERS)+')',expand=False).map(dictOTHERS)

##Search "Free From"


In [ ]:
dictSil = {'Silicone':'1'}

In [ ]:
df_launches['Free from'] = df_launches['Free from'].str.extract('('+'|'.join(dictSil)+')',expand=False).map(dictSil)

## Search ingredients and identify products that mache INCI



In [ ]:
df_inci.dropna(inplace=True)
df_inci['Produto'] = formatar_material(df_inci['Produto'])

In [ ]:
#Cria uma lista iterável dos ingredientes nos Produtos
df_inci['Ing'] = formatar_material(df_inci['Ingrediente']).str.split(', ').apply(set)

In [ ]:
#Conta a quantidade de ingredientes
df_inci.sort_values('Prioridade', ascending=True, inplace=True)

In [ ]:
 #Dicionário de palavras a remover da coluna Ingredientes no Mintel
 dictIng={' Andor ':',',
          ', ':',',
          ' ,':',',
          ' (And) ':',',
          }

In [ ]:
#Cria uma lista iterável das formulas do mintel
df_launches['Ingredients (Standard form)'] = df_launches['Ingredients (Standard form)'].astype(str)
df_launches['Ing'] = formatar_material(df_launches['Ingredients (Standard form)']).replace(dictIng,regex=True).dropna().str.split(',').apply(set)

In [ ]:
#Aplica a função Procura Produtos
df_launches[''] = df_launches['Ing'].apply(lambda formulacao:procura_produtos(formulacao, df_inci['Ing'], df_inci['Produto']))

## Remover Clientes  da planilha Mintel


In [ ]:
dictClientes = {'Devintex Cosméticos':'1', 'Devintex':'1','Procosa Produtos de Beleza':'1', 'Colgate-Palmolive':'1', 'Unilever':'1',
                'Savoy Indústria de Cosméticos':'1', 'Beiersdorf':'1', 'Johnson & Johnson':'1',
                'Procter & Gamble':'1','Química Amparo':'1', "Oréal":'1', 'Procosa':'1',
                'Unilever Brasil Industrial':'1', 'Botica Comercial Farmacêutica':'1'}
dictMarcas = { 'Boticário':'1', 'Siàge':'1', 'Natura ':'1', 'Avon ':'1', 'Eudora':'1',
              'Wella':'1', 'The Body Shop':'1', 'Quem Disse,':'1', 'Salon Line':'1', 'Maybelline':'1'}
df_launches['Clientes']= df_launches['Manufacturer'].str.extract('('+'|'.join(dictClientes)+')',expand=False).map(dictClientes)
df_launches['Clientes2'] = df_launches['Brand'].str.extract('('+'|'.join(dictMarcas)+')',expand=False).map(dictMarcas)
df_ = df_launches[df_launches['Clientes']=='1']
df_ = df_[df_['Clientes2']=='1']
df_.to_excel("LancClientes.xlsx")

In [ ]:
#Remove clientes  da base de tratamento
df_launches = df_launches[df_launches['Clientes']!='1']
df_launches = df_launches[df_launches['Clientes2']!='1']

## Cria Indice para junção

In [ ]:
df_launches.set_index(['Record ID'])
df_launches['Date Published'] = pd.to_datetime(df_launches['Date Published'], format='%Y-%m-%d')
df_launches['Quarter'] = df_launches['Date Published'].dt.quarter
df_launches['Year'] = df_launches['Date Published'].dt.year

In [ ]:
#Cria a coluna que irá indexar a mistura com a planilha do mintel
df_launches['Manufacturer']=df_launches['Manufacturer'].astype(str)
df_launches['KeyManuf'] = formatar_fabricantes(df_launches['Manufacturer'])

## Report: Lauches with Identified Material

In [ ]:
df_launches.columns

Index(['Record ID', 'Date Published', 'Product', 'Brand', 'Company', 'Market',
       'Category', 'Sub-Category', 'Product Description', 'Launch Type',
       'Manufacturer', 'Location of Manufacture',
       'Manufacturer Company Address', 'Ingredients (Standard form)',
       'Price in local currency', 'Primary Image Link', 'Pet', 'Convenience',
       'Minus', 'Functional', 'Natural', 'Plus', 'Positioning', 'Suitable for',
       'Demographic', 'Free from', 'Beauty enhancing',
       'Ethical & environmental', 'Product tested', 'State', 'Silicone', 'Ing',
       '', 'Clientes', 'Clientes2', 'Quarter', 'Year', 'KeyManuf'],
      dtype='object')

In [ ]:
col=['Quarter','Sub-Category', 'Product', 'Brand', 'Company', 'Manufacturer', 'KeyManuf', 'State', 'Free from', 'Price in local currency',
     'Silicone', '', 'Ing', 'Manufacturer Company Address',]

In [ ]:
df_launches[col].to_excel("Lancamentos Linha a Linha.xlsx")

## Generate a Reports on Manufacture activities

In [ ]:
transpor(df_launches, 'Category', '')

In [ ]:
#renomeia coluna categoria
df_launches.rename(columns={'Skincare':'Skin', 'Hair Products':'Hair',
                          'Soap & Bath Products':'Bath', 'Record ID': 'Total Lançamentos'},
                 inplace=True)

In [ ]:
#Cria tabela que destaca quais silicone da lista de INCI foram identificados, por categoria.
df_a = df_launches.groupby(['KeyManuf', 'State'])[['Hair', 'Skin', 'Colour Cosmetics', 'Bath']].apply(to_set)

In [ ]:
#cria tabela que lista o fabricante, o estado e o total de lançamentos, quais tem silicone,
#e quais tem o claim sem silicone.
df_b = df_launches.groupby(['KeyManuf', 'State']).agg({
                                            'Total Lançamentos': 'count',
                                             'Silicone':'count',
                                             'Free from':'count',
                                             })

In [ ]:
#Une tabelas anteriores
df_launches_fab = pd.concat([df_a, df_b], axis=1).reset_index()
#df_launches_fab.to_excel('Silicones by Category and Manufacture Launches.xlsx')

# Prepare Sales Report

In [ ]:
#Garantir que apenas serão considerados os clientes com vendas
df_dist['Year'].dropna(inplace=True)

In [ ]:
#formata coluna Material da tabela de vendas
df_dist['Material']= formatar_material(df_dist['Material'])
df_dist['Material'] = df_dist['Material'].astype(str)

In [ ]:
#formata coluna Fabricante da tabela de vendas
df_dist['KeyManuf'] = formatar_fabricantes(df_dist['Customer(at distrib)'])

In [ ]:
#agrupo os produtos em uma lista, por cliente, e soma volume de venda
df_dist_unic = df_dist.groupby(['KeyManuf','Distributor']).agg({'Year':'sum','Material':lambda x:list(set(x))}).reset_index()

In [ ]:
#tranforma lista acima em objeto iterável, e renomeia coluna
df_dist_unic['Material'] = df_dist_unic['Material'].apply(set)
df_dist_unic.rename(columns={'Material': 'Sales'}, inplace=True)

In [ ]:
df_dist_unic.drop_duplicates(subset=['KeyManuf', 'Distributor'], keep='first', inplace=True)
df_dist_unic.sample(5)

,KeyManuf,Distributor,Year,Sales
881,keef,FOCUS QUIMICA,0.145,"{Dm 6010, Cm 1000, Gb 150, Cm 740, Cm 040}"
877,kaori,FOCUS QUIMICA,0.106,"{Gb 150, Adm 9000, Dm 6010}"
1322,procerceramicosexportac,BRENNTAG,0.208,"{Silres Bs 16 280Kg, Silres Bs 1001}"
65,alfahair,FOCUS QUIMICA,0.040,{Gb 150}
349,cellerasa,BRENNTAG,0.160,{Hdk N20Pharma}


# Prepare Project Report

In [ ]:
#O nome do material deve estar em formato padrão
df_proj['Material']= formatar_material(df_proj['Material'])

In [ ]:
#força coluna virar string
df_proj['Customer'] = df_proj['Customer'].astype(str)

In [ ]:
#remove palavras chave
df_proj['KeyManuf'] = formatar_fabricantes(df_proj['Customer'])

In [ ]:
#agrupo todos os produtos em projeto em uma lista, por cliente
df_proj_unic = df_proj.groupby(['KeyManuf','Distributor']).agg({'Material':lambda x:list(set(x))}).reset_index()

In [ ]:
#Seta lista acima iterável, e modifica nome de coluna
df_proj_unic['Material'] = df_proj_unic['Material'].apply(set)
df_proj_unic.rename(columns={'Material': 'Project'}, inplace=True)

In [ ]:
#remove duplicados
df_proj_unic.drop_duplicates(subset=['KeyManuf', 'Distributor'], keep='first', inplace=True)
df_proj_unic.head(5)

,KeyManuf,Distributor,Project
0,&co,FOCUS QUIMICA,"{Tms 803, Hdk N20, Reg 1103 B}"
1,2k,EMBACAPS,"{Tms 803, Dm 065}"
2,3x,BRENNTAG,{Wacker 23}
3,a&d(sc),FOCUS QUIMICA,"{Pf 200, Pf 22, Adm 8301}"
4,a&drepresentacoes(sp),FOCUS QUIMICA,{Gb 7030}


# Group report Sales and Project

**#Ainda não funciona**

In [ ]:
df_proj_vend = df_proj_unic.merge(df_dist_unic, on=['KeyManuf', 'Distributor'], how='left')
df_proj_vend.head(5)

,KeyManuf,Distributor,Project,Year,Sales
0,&co,FOCUS QUIMICA,"{Tms 803, Hdk N20, Reg 1103 B}",NaN,NaN
1,2k,EMBACAPS,"{Tms 803, Dm 065}",0.302,"{Dm 0.65 20Kg, Dm 6010, Cm 040, Dm 1000, Eg 5}"
2,3x,BRENNTAG,{Wacker 23},0.990,"{Silfoam Se 39, Silfoam Sre, Liosil Hc 101, Dm..."
3,a&d(sc),FOCUS QUIMICA,"{Pf 200, Pf 22, Adm 8301}",NaN,NaN
4,a&drepresentacoes(sp),FOCUS QUIMICA,{Gb 7030},NaN,NaN


In [ ]:
df_vend_proj = df_dist_unic.merge(df_proj_unic, on=['KeyManuf', 'Distributor'], how='left')
df_vend_proj.head(5)

,KeyManuf,Distributor,Year,Sales,Project
0,,FOCUS QUIMICA,1.280,"{Cm 1306, Adm 9000, Adm 8105, Dm 350, Adm 8301...",NaN
1,1,BRENNTAG,0.020,"{E 10, Silfoam Sh}",NaN
2,2k,EMBACAPS,0.302,"{Dm 0.65 20Kg, Dm 6010, Cm 040, Dm 1000, Eg 5}","{Tms 803, Dm 065}"
3,3x,BRENNTAG,0.990,"{Silfoam Se 39, Silfoam Sre, Liosil Hc 101, Dm...",{Wacker 23}
4,a&d,FOCUS QUIMICA,8.500,"{Dm 6010, Adm 9000, Gb 7030, Dm 350, Pf 22, Cm...",NaN


In [ ]:
#df_proj_vend.to_excel('Projetos com Vendas.xlsx')
#df_vend_proj.to_excel('Vendas com Projetos.xlsx')

# Group all reports

In [ ]:
df_launches_vend_proj = df_launches_fab.merge(df_vend_proj, on='KeyManuf', how='left')
df_launches_vend_proj.head(5)

,KeyManuf,State,Hair,Skin,Colour Cosmetics,Bath,Total Lançamentos,Silicone,Free from,Distributor,Year,Sales,Project
0,,SP,{Dm},{},{},{},2,1,0,FOCUS QUIMICA,1.28,"{Cm 1306, Adm 9000, Adm 8105, Dm 350, Adm 8301...",NaN
1,&co,SP,{},"{Dm, Hdk N20, Hdk H, Es 3007}",{},{},2,2,0,NaN,NaN,NaN,NaN
2,5s,SP,NaN,NaN,NaN,NaN,2,0,0,NaN,NaN,NaN,NaN
3,a&agoldpharmainustria,MG,NaN,NaN,NaN,NaN,1,0,0,NaN,NaN,NaN,NaN
4,abboccato,SC,NaN,NaN,NaN,NaN,1,0,0,EMBACAPS,0.10,"{Eg 5, Adm 9000, Dm 6010, Cm 040}",NaN


In [ ]:
df_launches_vend_proj_2 = df_launches_fab.merge(df_proj_vend, on='KeyManuf', how='left')
df_launches_vend_proj_2.head(5)

,KeyManuf,State,Hair,Skin,Colour Cosmetics,Bath,Total Lançamentos,Silicone,Free from,Distributor,Project,Year,Sales
0,,SP,{Dm},{},{},{},2,1,0,NaN,NaN,NaN,NaN
1,&co,SP,{},"{Dm, Hdk N20, Hdk H, Es 3007}",{},{},2,2,0,FOCUS QUIMICA,"{Tms 803, Hdk N20, Reg 1103 B}",NaN,NaN
2,5s,SP,NaN,NaN,NaN,NaN,2,0,0,NaN,NaN,NaN,NaN
3,a&agoldpharmainustria,MG,NaN,NaN,NaN,NaN,1,0,0,NaN,NaN,NaN,NaN
4,abboccato,SC,NaN,NaN,NaN,NaN,1,0,0,NaN,NaN,NaN,NaN


In [93]:
df_launches_vend_proj_3 = pd.concat([df_launches_vend_proj, df_launches_vend_proj_2])
df_launches_vend_proj_3.drop_duplicates(subset=['KeyManuf','State','Distributor'],inplace=True)
df_launches_vend_proj_3.reset_index

In [ ]:
df_launches_vend_proj_3.rename(columns={'': ' Prod Poss','Customer(at distrib)':'Customer Sales','Distributor':'Dist. Sales',
                                    'Sales': 'Prod Sales','Year': 'Sales Volume', 'Customer': 'Customer Proj Fab',
                                    'Distributor':'Dist Proj Fab', 'Project':'Prod Proj Fab',
                                    'Fase':'Fase Proj Fab',
                                         },
                                inplace=True)

# Generate a Report will all information

In [ ]:
df_launches_vend_proj_3.to_excel('Final.xlsx')